# **Architettura e Design Patterns per Survival Game in Unity - Riassunto Concettuale**

## **1. Filosofia Progettuale: Perché Questo Approccio**
Questo progetto dimostra **design pattern applicati a giochi reali**, non solo codice funzionale. L'obiettivo è creare un'architettura:
- **Modulare**: ogni sistema è indipendente
- **Scalabile**: si aggiungono feature senza riscrivere
- **Manutenibile**: codice chiaro dopo 6 mesi
- **Professionale**: pattern usati nell'industria

---

## **2. Hierarchy Design Pattern: La Separazione delle Responsabilità**

### **Pattern: Transform Hierarchy Separation**
```
Player (ROOT)           → Logica, Fisica, Posizione mondiale
├─ CameraContainer      → Rotazione verticale (locale)
│   └─ Main Camera      → Rendering, FOV
├─ MeshContainer        → Modello 3D, animazioni
└─ (Componenti invisibili) → Collider, Rigidbody
```

### **Perché è Fondamentale:**
```csharp
// SBAGLIATO: Tutto insieme
transform.Rotate(mouseY, mouseX, 0); 
// Camera ruota con il collider → clipping strano

// CORRETTO: Separato
cameraContainer.Rotate(mouseY, 0, 0);  // Solo camera
transform.Rotate(0, mouseX, 0);         // Solo corpo
```
**Vantaggi tecnici:**
1. **Collisioni stabili**: il collider non ruota con la camera
2. **Animazioni pulite**: il modello può essere sostituito
3. **Networking friendly**: solo il ROOT viene sincronizzato
4. **VR/Third-person ready**: cambio camera senza modifiche

---

## **3. Input System Architecture: Event-Driven Design**

### **Perché NON usare Input.GetKey():**
- Hardcoded keys (no rebinding)
- Spaghetti code (if/else everywhere)
- No controller support nativo
- Difficile da testare

### **Pattern Corretto: Input Action Assets**
```csharp
// PlayerInputHandler.cs - Decoupling completo
public class PlayerInputHandler : MonoBehaviour
{
    // 1. Riferimento alle azioni
    [SerializeField] private InputActionReference moveAction;
    [SerializeField] private InputActionReference lookAction;
    
    // 2. Eventi esposti al controller
    public event Action<Vector2> OnMovePerformed;
    public event Action<Vector2> OnLookPerformed;
    
    private void OnEnable()
    {
        // 3. Binding pulito
        moveAction.action.performed += ctx => 
            OnMovePerformed?.Invoke(ctx.ReadValue<Vector2>());
            
        lookAction.action.performed += ctx =>
            OnLookPerformed?.Invoke(ctx.ReadValue<Vector2>());
    }
    
    // 4. Cleanup automatico
    private void OnDisable() => DisableAll();
}
```

**Flusso dati ideale:**
```
Input System → InputHandler (eventi) → PlayerController (logica) → Rigidbody (fisica)
```

---

## **4. Physics-Based Movement: Perché Rigidbody e Non CharacterController**

### **Comparison Tecnica:**
| **CharacterController** | **Rigidbody + CapsuleCollider** |
|------------------------|--------------------------------|
| Movimento "finto" | Fisica reale |
| Collisioni semplici | Collisioni realistiche |
| No force feedback | Forze, esplosioni, physics interaction |
| Difficile per multiplayer | Predicibile, sync-friendly |
| Limitato per survival | Estensibile (acqua, vento, slidi) |

### **Implementazione Corretta:**
```csharp
public class PhysicsBasedMovement : MonoBehaviour
{
    [SerializeField] private float moveSpeed = 5f;
    [SerializeField] private float groundDrag = 5f;
    [SerializeField] private float airMultiplier = 0.5f;
    
    private Rigidbody rb;
    private bool isGrounded;
    
    private void FixedUpdate()
    {
        // 1. Calcolo direzione (local space → world space)
        Vector3 moveDirection = 
            transform.forward * input.y + 
            transform.right * input.x;
            
        // 2. Applica velocità in base allo stato
        if (isGrounded)
        {
            rb.AddForce(moveDirection * moveSpeed, ForceMode.Force);
            rb.drag = groundDrag;
        }
        else
        {
            rb.AddForce(moveDirection * moveSpeed * airMultiplier, ForceMode.Force);
            rb.drag = 0;
        }
        
        // 3. Limita velocità orizzontale
        Vector3 flatVel = new Vector3(rb.velocity.x, 0, rb.velocity.z);
        if (flatVel.magnitude > moveSpeed)
        {
            Vector3 limitedVel = flatVel.normalized * moveSpeed;
            rb.velocity = new Vector3(limitedVel.x, rb.velocity.y, limitedVel.z);
        }
    }
}
```

**Key Points:**
- Usa `FixedUpdate()` per fisica
- `ForceMode.Force` per movimento smooth
- Drag per controllo "weight"
- Velocity clamping per evitare accelerazione infinita

---

## **5. Ground Detection System: Robust Raycast Pattern**

### **Problema del Single Raycast:**
- Fails on edges (false negative)
- Fails on slopes (false positive)
- Unreliable for jumping

### **Soluzione: Multi-Raycast Array**
```csharp
public class AdvancedGroundCheck : MonoBehaviour
{
    [SerializeField] private LayerMask groundLayer;
    [SerializeField] private float checkDistance = 0.15f;
    [SerializeField] private float checkRadius = 0.3f;
    
    private bool IsGrounded()
    {
        // 1. Raycast centrale (primario)
        if (Physics.Raycast(transform.position, Vector3.down, 
            checkDistance + 0.1f, groundLayer))
            return true;
        
        // 2. Raycast a croce (backup)
        Vector3[] offsets = new Vector3[]
        {
            new Vector3(checkRadius, 0, 0),
            new Vector3(-checkRadius, 0, 0),
            new Vector3(0, 0, checkRadius),
            new Vector3(0, 0, -checkRadius)
        };
        
        foreach (Vector3 offset in offsets)
        {
            if (Physics.Raycast(transform.position + offset, Vector3.down,
                checkDistance, groundLayer))
                return true;
        }
        
        // 3. SphereCast per slope detection (opzionale)
        if (Physics.SphereCast(transform.position, checkRadius * 0.5f,
            Vector3.down, out RaycastHit hit, checkDistance, groundLayer))
            return true;
            
        return false;
    }
}
```

---

## **6. Needs System: Data-Oriented Design**

### **Architettura a Strati:**
```
Need (SO/Class) → PlayerNeeds (Manager) → UI_Needs (Presenter)
```

```csharp
[System.Serializable]
public class Need
{
    public string needName;
    public float currentValue;
    public float maxValue;
    
    [Header("Rates")]
    public float decayRate = 0.1f;  // Perdita al secondo
    public float regenRate = 0.5f;  // Rigenerazione
    
    [Header("Thresholds")]
    public float criticalThreshold = 20f;
    public float lowThreshold = 50f;
    
    // Eventi per UI/Feedback
    public event Action<float> OnValueChanged;
    public event Action OnCritical;
    public event Action OnLow;
    
    public void Update(float deltaTime)
    {
        float oldValue = currentValue;
        currentValue -= decayRate * deltaTime;
        currentValue = Mathf.Clamp(currentValue, 0, maxValue);
        
        // Notifica solo se cambiamento significativo
        if (Mathf.Abs(oldValue - currentValue) > 0.1f)
        {
            OnValueChanged?.Invoke(currentValue);
            
            if (currentValue <= criticalThreshold && oldValue > criticalThreshold)
                OnCritical?.Invoke();
        }
    }
}

// PlayerNeeds.cs - Coordinatore
public class PlayerNeeds : MonoBehaviour, IDamageable
{
    public Need health;
    public Need hunger;
    public Need thirst;
    public Need stamina;
    
    private Need[] allNeeds;
    
    private void Start()
    {
        allNeeds = new Need[] { health, hunger, thirst, stamina };
        
        // Setup interdipendenze
        hunger.OnCritical += () => health.UpdateValue(-10f);
        thirst.OnCritical += () => health.UpdateValue(-15f);
    }
    
    private void Update()
    {
        float deltaTime = Time.deltaTime;
        
        foreach (Need need in allNeeds)
        {
            need.Update(deltaTime);
        }
        
        // Effetto fame/sete sulla stamina
        stamina.regenRate = (hunger.currentValue > 50f && thirst.currentValue > 50f) 
            ? 2f : 0.5f;
    }
}
```

---

## **7. Damage System: Interface Pattern per Loose Coupling**

### **Perché le Interface sono Cruciali:**
```csharp
// IDamageable.cs - Contratto universale
public interface IDamageable
{
    void TakeDamage(DamageData damageData);
    void Heal(float amount);
    bool IsAlive { get; }
    float CurrentHealth { get; }
}

// DamageData.cs - Struttura estensibile
public struct DamageData
{
    public float amount;
    public DamageType type;
    public Vector3 hitPoint;
    public Vector3 hitNormal;
    public GameObject instigator;
    
    public enum DamageType
    {
        Physical,
        Fire,
        Poison,
        Hunger,
        Thirst
    }
}

// CactusDamage.cs - Implementazione semplice
public class CactusDamage : MonoBehaviour
{
    [SerializeField] private float damagePerSecond = 5f;
    [SerializeField] private float damageInterval = 0.5f;
    
    private HashSet<IDamageable> damageablesInRange = new();
    
    private void OnTriggerEnter(Collider other)
    {
        if (other.TryGetComponent<IDamageable>(out IDamageable damageable))
        {
            damageablesInRange.Add(damageable);
            StartCoroutine(DamageRoutine());
        }
    }
    
    private IEnumerator DamageRoutine()
    {
        while (damageablesInRange.Count > 0)
        {
            // Crea dati danno consistenti
            DamageData damage = new DamageData
            {
                amount = damagePerSecond * damageInterval,
                type = DamageData.DamageType.Physical,
                hitPoint = transform.position,
                instigator = gameObject
            };
            
            foreach (IDamageable d in damageablesInRange.ToList())
            {
                if (d != null && d.IsAlive)
                    d.TakeDamage(damage);
                else
                    damageablesInRange.Remove(d);
            }
            
            yield return new WaitForSeconds(damageInterval);
        }
    }
}
```

**Vantaggi di questo pattern:**
1. **Estensibile**: aggiungi nuovi tipi di danno senza modificare il cactus
2. **Testabile**: mock interface per unit test
3. **Clean**: il cactus non conosce Player/Enemy/NPC

---

## **8. UI Architecture: Event-Driven UI Pattern**

### **Pattern: Observer + UnityEvents**
```csharp
// NeedUI.cs - Presenter puro
public class NeedUI : MonoBehaviour
{
    [SerializeField] private Image fillImage;
    [SerializeField] private Image background;
    [SerializeField] private Color normalColor = Color.green;
    [SerializeField] private Color criticalColor = Color.red;
    
    private Need targetNeed;
    
    public void Bind(Need need)
    {
        if (targetNeed != null)
        {
            targetNeed.OnValueChanged -= UpdateUI;
            targetNeed.OnCritical -= OnCriticalState;
        }
        
        targetNeed = need;
        
        targetNeed.OnValueChanged += UpdateUI;
        targetNeed.OnCritical += OnCriticalState;
        
        UpdateUI(need.currentValue);
    }
    
    private void UpdateUI(float value)
    {
        fillImage.fillAmount = value / targetNeed.maxValue;
        
        // Interpolazione colore
        float t = value / targetNeed.maxValue;
        fillImage.color = Color.Lerp(criticalColor, normalColor, t);
    }
    
    private void OnCriticalState()
    {
        // Animazione di warning
        StartCoroutine(FlashWarning());
    }
    
    private void OnDestroy()
    {
        if (targetNeed != null)
        {
            targetNeed.OnValueChanged -= UpdateUI;
            targetNeed.OnCritical -= OnCriticalState;
        }
    }
}
```

---

## **9. Checklist per Progetto Serio**

### **Cose da Implementare SEMPRE:**
- [ ] **Input System** nuovo (non Input Manager)
- [ ] **Layer Matrix** configurata correttamente
- [ ] **Physics Layers** per raycast selettivi
- [ ] **Tag system** per identificazione rapida
- [ ] **Event system** per decoupling
- [ ] **Serializable classes** per dati inspector-friendly
- [ ] **Editor scripts** per workflow migliorato

### **Performance Considerations:**
```csharp
// Ottimizzazioni immediate:
1. Cache i GetComponent<T>()
2. Usa Coroutines invece di Update per logica non-critica
3. Pooling per oggetti frequenti (proiettili, effetti)
4. LOD per sistemi needs (update rate differenziato)
```

### **Architettura Next-Level:**
```csharp
// 1. ScriptableObject per configurazione
[CreateAssetMenu]
public class PlayerConfig : ScriptableObject
{
    public float moveSpeed;
    public float jumpForce;
    public NeedSettings healthSettings;
    // ...
}

// 2. State Machine per comportamento complesso
public class PlayerStateMachine : MonoBehaviour
{
    public IPlayerState CurrentState { get; private set; }
    
    public void ChangeState(IPlayerState newState)
    {
        CurrentState?.Exit();
        CurrentState = newState;
        CurrentState?.Enter();
    }
}

// 3. Dependency Injection leggera
public class PlayerContext
{
    public PlayerConfig Config;
    public InputHandler Input;
    public NeedsSystem Needs;
    // ...
}
```

---

## **10. Key Takeaways per Portfolio**

### **Cosa Mostrare:**
1. **Hierarchy design** con ROOT pattern
2. **Input System** con rebinding
3. **Physics movement** con slope handling
4. **Needs system** con interdipendenze
5. **Interface-based damage** system
6. **Event-driven UI** con animazioni

### **Cosa Evitare:**
- Singleton globali
- FindObjectOfType() in Update
- Input.GetKey() hardcoded
- Public fields senza [SerializeField]
- Coroutine senza stop condition

---

## **11. Refactoring Suggeriti per V2**

### **PlayerController Refactored:**
```csharp
public class AdvancedPlayerController : MonoBehaviour
{
    // Dependencies (iniettate o serializzate)
    [SerializeField] private PlayerConfig config;
    [SerializeField] private InputHandler inputHandler;
    [SerializeField] private GroundDetector groundDetector;
    
    // Components
    private Rigidbody rb;
    private PlayerStateMachine stateMachine;
    
    private void Awake()
    {
        rb = GetComponent<Rigidbody>();
        stateMachine = new PlayerStateMachine();
        
        // Setup stati
        var groundedState = new GroundedState(this, config);
        var airState = new AirborneState(this, config);
        var swimmingState = new SwimmingState(this, config);
        
        stateMachine.Initialize(groundedState);
    }
    
    private void Update()
    {
        stateMachine.CurrentState?.Update();
    }
    
    private void FixedUpdate()
    {
        stateMachine.CurrentState?.FixedUpdate();
    }
}

// State interface
public interface IPlayerState
{
    void Enter();
    void Update();
    void FixedUpdate();
    void Exit();
}
```

---
